# **Retail Store's Sales Forecasting**

# **Store's/Dept's Weekly Sales Prediction**

In [ ]:
# Required packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import skew
import sklearn.metrics as metrics
import sklearn.model_selection as model_selection
import pickle

# **Data Loading**

In [ ]:
store_ds=pd.read_csv('/content/stores_data_set.csv')
print(store_ds.shape)
store_ds.head(3)

In [ ]:
sale_ds=pd.read_csv('/content/sales_data_set.csv')
print(sale_ds.shape)
sale_ds.head(3)

In [ ]:
feature_ds=pd.read_csv('/content/Features_data_set.csv')
print(feature_ds.shape)
feature_ds.head(3)

In [ ]:
feature_ds=feature_ds.sort_values(by=['Date','Store'])  # 183 * 45
feature_ds

# **EDA**

# **Data Analysis**

**split column**

In [ ]:
# adding new col week as its a weekly prediction
feature_ds['Date']=pd.to_datetime(feature_ds['Date'])
feature_ds['Week']=feature_ds.Date.dt.isocalendar().week
feature_ds['Year']=feature_ds.Date.dt.year
feature_ds.drop('Date',axis=1,inplace=True)

**Filter data from 2010 to 2012**

In [ ]:
# in store dataset 2010 to 2012 data only there.
feature_df=feature_ds.query('Year!=2013').copy()

In [ ]:
sale_df=sale_ds.copy()
sale_df['Total_Sales']=sale_df.groupby(['Store','Date','IsHoliday']).Weekly_Sales.transform('sum')

In [ ]:
sale_ds['Date']=pd.to_datetime(sale_df['Date'])
sale_ds['Week']=sale_ds.Date.dt.isocalendar().week
sale_ds['Year']=sale_ds.Date.dt.year
sale_ds.sort_values(by=['Dept','Date','IsHoliday'],ascending=True)
sale_ds.drop('Date',axis=1,inplace=True)

In [ ]:
# Sum up all sales for each department
sale_df=sale_ds.copy()
sale_grouped=sale_df.groupby(['Store','IsHoliday','Week','Year'])

sale_df=sale_grouped['Weekly_Sales'].sum()
sale_df=sale_df.reset_index(name='Total_sales')
sale_df

**Merge dataframes**

In [ ]:
# join both dataframes
merge_sale_df=pd.merge(sale_df,feature_df,how='inner',on=['Store','IsHoliday','Week','Year'])

In [ ]:
# checking for duplicates
merge_sale_df1=merge_sale_df[merge_sale_df.duplicated(subset=['Store','IsHoliday','Week','Year','Total_sales'],keep='first')==True].sort_values(by=['Store','Week'],ascending=True)
merge_sale_df1

In [ ]:
# delete duplicates
merge_sale_df.drop_duplicates(subset=['Store', 'IsHoliday', 'Week', 'Year', 'Total_sales'], keep='first', inplace=True)

In [ ]:
store_sales_df=merge_sale_df[['Store','Week','Year','IsHoliday','Temperature','Fuel_Price','CPI','Unemployment','MarkDown1','MarkDown2','MarkDown3','MarkDown4','MarkDown5','Total_sales']].copy()
store_sales_df

In [ ]:
# this dataset for dept weekly sales prediction
dept_sale_df=sale_ds.copy()
dept_sale_df

# **Store's Weekly Sales Prediction**

# **Data Cleaning/ Pre Processing**

**Finding null values**

In [ ]:
# Size
print("Size: ", store_sales_df.shape)

# To check the column names
print("\nColumn list: ",store_sales_df.columns)

# column information
print("\nInfo: ")
store_sales_df.info()

In [ ]:
null_data = store_sales_df.isnull().mean()*100
print("Null values\n")
pd.DataFrame({
    "column_name": null_data.index,
    "Null values":null_data.values
}).sort_values("Null values",ascending=False)

In [ ]:
# description of the column
print("Description\n")
store_sales_df.describe()

**Imputation**

In [ ]:
#Fill the null values
store_sales_df['MarkDown1'].fillna(0, inplace=True)
store_sales_df['MarkDown2'].fillna(0, inplace=True)
store_sales_df['MarkDown3'].fillna(0, inplace=True)
store_sales_df['MarkDown4'].fillna(0, inplace=True)
store_sales_df['MarkDown5'].fillna(0, inplace=True)

**Checking Duplicates**

In [ ]:
print("Total Duplicated datapoints: ",store_sales_df.duplicated().sum())

**Number of "0" data availablity - sparcity**

In [ ]:
# full row zero
col_with_zero = []
for i in store_sales_df.columns:
    perc_zero = (store_sales_df[i]==0).mean()*100
    col_with_zero.append((i,perc_zero))

zero_percent = pd.DataFrame(col_with_zero,columns=['column_name','zero_percentage']).sort_values("zero_percentage",ascending=False)
zero_percent

**Datatype Change**

In [ ]:
store_sales_df['IsHoliday'] = store_sales_df['IsHoliday'].astype(int)
store_sales_df['Week'] = store_sales_df['Week'].astype(int)

In [ ]:
store_sales_df.info()

In [ ]:
unique_values = store_sales_df['IsHoliday'].unique()
print(unique_values)

# **Statistical Analysis**

**Finding Outliers**

**IQR (Interquartile Range Method)**

In [ ]:
def out_iqr(df , column):
    global lower,upper
    q25, q75 = np.quantile(df[column], 0.25), np.quantile(df[column], 0.75)
    # calculate the IQR
    iqr = q75 - q25
    cut_off = iqr * 1.5
    lower, upper = q25 - cut_off, q75 + cut_off
    df1 = df[df[column] > upper]
    df2 = df[df[column] < lower]
    return print(f'Total number of outliers in {column} :', df1.shape[0]+ df2.shape[0])

In [ ]:
iqr_ds=store_sales_df.copy()
for i in iqr_ds.columns:
  out_iqr(iqr_ds,i)

**outliers visualization  z-scores**

In [ ]:
num_rows = 5
num_cols = 3
total_plots = len(store_sales_df.columns)

# Calculate the number of plots per figure
plots_per_figure = num_rows * num_cols

for i in range(0, total_plots, plots_per_figure):
    plt.figure(figsize=(15, 10))
    for j, column_name in enumerate(store_sales_df.columns[i:i+plots_per_figure]):
        plt.subplot(num_rows, num_cols, j + 1)
        z_scores = (store_sales_df[column_name] - store_sales_df[column_name].mean()) / store_sales_df[column_name].std()

        threshold = 3

        plt.scatter(x=range(len(store_sales_df[column_name])), y=store_sales_df[column_name],
                    c=np.where(np.abs(z_scores) > threshold, 'red', 'blue'))

        plt.title(f'{column_name} Outliers\n')

    plt.tight_layout()
    plt.show()


**Skewness**

In [ ]:
before_transform = store_sales_df.skew()

In [ ]:
num_rows = 5
num_cols = 3
num_columns = len(store_sales_df.columns)
fig, axs = plt.subplots(num_rows, num_cols, figsize=(15, 20))
axs = axs.flatten()

for i, column_name in enumerate(store_sales_df.columns):

    skewness = skew(store_sales_df[column_name])
    sns.histplot(store_sales_df[column_name], bins=10, kde=True, color='cyan', ax=axs[i])
    axs[i].axvline(np.mean(store_sales_df[column_name]), color='red', linestyle='dashed', linewidth=2, label='Mean')
    axs[i].axvline(np.median(store_sales_df[column_name]), color='blue', linestyle='dashed', linewidth=2, label='Median')

    if -1 <= skewness <= 1:
        res="Symmetric distribution."
    elif skewness > 1:
      res="Right-skewed distribution."
    else:
        res="Left-skewed distribution."
    axs[i].set_title(f'Histogram for {column_name}\nSkewness: {skewness:.2f} \n {res}')
    axs[i].legend()

plt.tight_layout()
plt.show()


In [ ]:
store_sales_df

# **Feature Engineering**

**Log Transformation**

In [ ]:
epsilon = 1
store_sales_df1=store_sales_df.copy()
store_sales_df['MarkDown1'] = np.log(store_sales_df['MarkDown1']+ epsilon)
store_sales_df['MarkDown2'] = np.log(store_sales_df['MarkDown2']+ epsilon)
store_sales_df['MarkDown3'] = np.log(store_sales_df['MarkDown3']+ epsilon)
store_sales_df['MarkDown4'] = np.log(store_sales_df['MarkDown4']+ epsilon)
store_sales_df['MarkDown5'] = np.log(store_sales_df['MarkDown5']+ epsilon)
store_sales_df['Temperature'] = np.log(store_sales_df['Temperature'])
store_sales_df['Total_sales'] = np.log(store_sales_df['Total_sales'])
store_sales_df['Unemployment'] = np.log(store_sales_df['Unemployment'])

In [ ]:
# fill -ve values with mean
store_sales_df['MarkDown1']=store_sales_df['MarkDown1'].apply(lambda x: store_sales_df['MarkDown1'].mean() if x <= 0 else x)
store_sales_df['MarkDown2']=store_sales_df['MarkDown2'].apply(lambda x: store_sales_df['MarkDown2'].mean() if x <= 0 else x)
store_sales_df['MarkDown3']=store_sales_df['MarkDown3'].apply(lambda x: store_sales_df['MarkDown3'].mean() if x <= 0 else x)
store_sales_df['Temperature'] = store_sales_df['Temperature'].apply(lambda x: store_sales_df['Temperature'].mean() if x <= 0 else x)

In [ ]:
store_sales_df['MarkDown1'].fillna(store_sales_df['MarkDown1'].mean(),inplace=True)
store_sales_df['MarkDown2'].fillna(store_sales_df['MarkDown2'].mean(), inplace=True)
store_sales_df['MarkDown3'].fillna(store_sales_df['MarkDown3'].mean(),inplace=True)
store_sales_df['MarkDown4'].fillna(store_sales_df['MarkDown4'].mean(), inplace=True)
store_sales_df['MarkDown5'].fillna(store_sales_df['MarkDown5'].mean(), inplace=True)
store_sales_df['Temperature'].fillna(store_sales_df['Temperature'].mean(), inplace=True)

In [ ]:
store_sales_df[store_sales_df['MarkDown1']==np.inf]== store_sales_df['MarkDown1'].mean()

In [ ]:
after_transform = store_sales_df.skew()

In [ ]:
print("Before Transformation\n",before_transform)
print("After Transformation\n",after_transform)

# **Feature Selection**

In [ ]:
corr_data = store_sales_df.corr()
plt.figure(figsize=(16,6))
sns.heatmap(corr_data,annot=True,cmap="coolwarm",fmt=".2f")

# **Model Selection**

In [ ]:
y= store_sales_df['Total_sales']
x = store_sales_df.drop('Total_sales', axis =1)

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

In [ ]:
from sklearn.ensemble import RandomForestRegressor
store_model_rfr = RandomForestRegressor(random_state=40).fit(x_train, y_train)
y_pred= store_model_rfr.predict(x_test)
print("MAE:",metrics.mean_absolute_error(y_test, y_pred))
mse=metrics.mean_squared_error(y_test, y_pred)
rmse=np.sqrt(mse)
print("MSE:",mse)
print("RMSE:",rmse)
print("R2 Score:",metrics.r2_score(y_test, y_pred))

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
model_gbr = GradientBoostingRegressor().fit(x_train, y_train)
y_pred=model_gbr.predict(x_test)
print("MAE:",metrics.mean_absolute_error(y_test, y_pred))
mse=metrics.mean_squared_error(y_test, y_pred)
rmse=np.sqrt(mse)
print("MSE:",mse)
print("RMSE:",rmse)
print("R2 Score:",metrics.r2_score(y_test, y_pred))

In [ ]:
from sklearn.tree import  DecisionTreeRegressor
model_dtr = DecisionTreeRegressor().fit(x_train, y_train)
y_pred=model_dtr.predict(x_test)
print("MAE:",metrics.mean_absolute_error(y_test, y_pred))
mse=metrics.mean_squared_error(y_test, y_pred)
rmse=np.sqrt(mse)
print("MSE:",mse)
print("RMSE:",rmse)
print("R2 Score:",metrics.r2_score(y_test, y_pred))

In [ ]:
x_test

**Test Data**

In [ ]:
# Sample data1
test_data=np.array([[1,1,2010,0,1.45,2.63,211.67,0.722,0.80,0.46,0.433,0,0]])
y_pred=store_model_rfr.predict(test_data)
y_pred[0]

In [ ]:
store_sales_df.head(3)

In [ ]:
x

In [ ]:
store_sales_df.tail(3)

In [ ]:
# Sample data2
test_data=np.array([[45,52,2011,1,1.2,3.38,189.06,0.76,2.2,2.45,1.8,1.92,2.03]])
y_pred=store_model_rfr.predict(test_data)
y_pred[0]

In [ ]:
# write pickle file  for weekly sale prediction
with open('/content/store_weekly_sale.pkl', 'wb') as f:
    pickle.dump(store_model_rfr, f)

# **Dept's weekly sales prediction**

# **PreProcessing**

In [ ]:
null_data = dept_sale_df.isnull().mean()*100
pd.DataFrame({
    "column_name": null_data.index,
    "Null values":null_data.values
}).sort_values("Null values",ascending=False)

In [ ]:
print("Total Duplicated datapoints: ",dept_sale_df.duplicated().sum())

In [ ]:
dept_sale_df.drop_duplicates(subset=['Store', 'IsHoliday', 'Week', 'Year', 'Dept','Weekly_Sales'], keep='first', inplace=True)

In [ ]:
# full row zero
col_with_zero = []
for i in dept_sale_df.columns:
    perc_zero = (dept_sale_df[i]==0).mean()*100
    col_with_zero.append((i,perc_zero))

zero_percent = pd.DataFrame(col_with_zero,columns=['column_name','zero_percentage']).sort_values("zero_percentage",ascending=False)
zero_percent

In [ ]:
dept_sale_df.info()

**Datatype Change**

In [ ]:
dept_sale_df['IsHoliday']=dept_sale_df['IsHoliday'].astype('int')
dept_sale_df['Week']=dept_sale_df['Week'].astype('int')

In [ ]:
dept_sale_df.shape

In [ ]:
dept_sale_df.describe()

In [ ]:
dept_sale_df.skew()

In [ ]:
for i in dept_sale_df.columns:
    skewness = dept_sale_df[i].skew()
    if -1 <= skewness <= 1:
        res="Symmetric distribution."
    elif skewness > 1:
      res="Right-skewed distribution."
    else:
        res="Left-skewed distribution."
    print(i," skewness:",skewness," ",res)

In [ ]:
dept_sale_df.std()

In [ ]:
dept_sale_df.hist(figsize=(10,10))

# **Feature Selection**

In [ ]:
corr_data = dept_sale_df.corr()
plt.figure(figsize=(16,6))
sns.heatmap(corr_data,annot=True,cmap="coolwarm",fmt=".2f")

In [ ]:
# Finding the outliers using IQR
def findOutliers(data, col):
	Q3 = np.quantile(data[col], 0.75)
	Q1 = np.quantile(data[col], 0.25)
	IQR = Q3 - Q1

	print("IQR value for column %s is: %s" % (col, IQR))

	lower_range = Q1 - 1.5 * IQR
	upper_range = Q3 + 1.5 * IQR
	x=np.where((data[col] > upper_range) | (data[col] < lower_range))
	print(col,":",lower_range, upper_range,"Outliers:",x[0])
for i in dept_sale_df.columns:
	findOutliers(dept_sale_df, i)

# **Model Selection**

In [ ]:
y= dept_sale_df['Weekly_Sales']
x = dept_sale_df.drop('Weekly_Sales', axis =1)

In [ ]:
# Splitting data into train(80%) and test(20%)
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=20)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
model_gbr = GradientBoostingRegressor().fit(x_train, y_train)
y_pred=model_gbr.predict(x_test)
print("MAE:",metrics.mean_absolute_error(y_test, y_pred))
mse=metrics.mean_squared_error(y_test, y_pred)
rmse=np.sqrt(mse)
print("MSE:",mse)
print("RMSE:",rmse)
print("R2 Score:",metrics.r2_score(y_test, y_pred))

In [ ]:
from sklearn.ensemble import RandomForestRegressor
dept_model_rfr = RandomForestRegressor().fit(x_train, y_train)
y_pred=dept_model_rfr.predict(x_test)
print("MAE:",metrics.mean_absolute_error(y_test, y_pred))
mse=metrics.mean_squared_error(y_test, y_pred)
rmse=np.sqrt(mse)
print("MSE:",mse)
print("RMSE:",rmse)
print("R2 Score:",metrics.r2_score(y_test, y_pred))

In [ ]:
from sklearn.tree import  DecisionTreeRegressor
model_dtr = DecisionTreeRegressor().fit(x_train, y_train)
y_pred=model_dtr.predict(x_test)
print("MAE:",metrics.mean_absolute_error(y_test, y_pred))
mse=metrics.mean_squared_error(y_test, y_pred)
rmse=np.sqrt(mse)
print("MSE:",mse)
print("RMSE:",rmse)
print("R2 Score:",metrics.r2_score(y_test, y_pred))

In [ ]:
dept_sale_df.head(3)

**Test Data**

In [ ]:
test_data=np.array([[1,1,0,17,2010]])
y_pred=dept_model_rfr.predict(test_data)
y_pred[0]

In [ ]:
dept_sale_df.tail(3)

In [ ]:
# Sample data2
test_data=np.array([[45,98,0,43,2012]])
y_pred=dept_model_rfr.predict(test_data)
y_pred[0]

In [ ]:
# write pickle file  for Department wise sale prediction
with open('/content/dept_sale.pkl', 'wb') as f:
    pickle.dump(dept_model_rfr, f)

In [ ]:
store_sales_df.head(3)

# **Streamlit App**

In [ ]:
%%writefile app.py
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.metrics as metrics
import sklearn.model_selection as model_selection
import pickle
import streamlit as st
from PIL import Image
from datetime import datetime
from scipy.stats import skew


#streamlit  page setting
icon = Image.open("store.jpg")
st.set_page_config(page_title= "Retail Store - Kavitha",
                page_icon= icon,
                layout= "wide",
                initial_sidebar_state= "expanded",
                )

st.subheader(":blue[Retail Store Sales Forecasting]")
tab1,tab2,tab3=st.tabs([":blue[Department]",":blue[Store]",":blue[About]"])

with tab1:
  col1,col2,col3=st.columns(3)
  with col1:
    txt_d_store=st.number_input("Enter Store No")
    txt_dept=st.number_input("Enter Dept No")
  with col2:
    txt_date=st.text_input("Enter the Date",datetime.today().strftime("%d/%m/%Y"))
    dt1=datetime.strptime(txt_date, "%d/%m/%Y")
    dept_week=dt1.isocalendar().week
    dept_year=dt1.year
  with col3:
    txt_holiday=st.selectbox("Holiday in date entered week", ("True","False"))
    if txt_holiday=="True":
      holiday=0
    else:
      holiday=1



  if st.button("Predict Weekly Sales", key="Department's WeeklySales"):
      # load the regression pickle model
      with open('/content/dept_sale.pkl', 'rb') as f:
          model_dpt = pickle.load(f)

      # make array for all user input values in required order for model prediction
      user_data = np.array([[int(txt_d_store),int(txt_dept),int(holiday),int(dept_week),int(dept_year)]])

      # model predict the Department sales based on user input
      y_pred = model_dpt.predict(user_data)
      d_weekly_sale = y_pred[0]

      # round the value with 2 decimal point
      d_weekly_sale = round(d_weekly_sale, 2)
      st.write("Department weekly sales: ", d_weekly_sale)

with tab2:
  col4,col5,col6=st.columns(3)
  with col4:
    txt_store=st.number_input("Store No")
    txt_date1=st.text_input("Date" ,datetime.today().strftime("%d/%m/%Y"))
    txt_holiday1=st.selectbox("Holiday", ("True","False"))
    txt_temp=st.number_input("Temperature in celcius")

    if txt_holiday1=="True":
      week_holiday=0
    else:
      week_holiday=1
  with col5:
    txt_fuel_price=st.number_input("Fuel Price")
    txt_CPI=st.number_input("CPI")
    txt_unemployment=st.number_input("Unemployment")
    txt_Markdown1=st.number_input("MarkDown1")


  with col6:
    txt_Markdown2=st.number_input("MarkDown2")
    txt_Markdown3=st.number_input("MarkDown3")
    txt_Markdown4=st.number_input("MarkDown4")
    txt_Markdown5=st.number_input("MarkDown5")



    dt2=datetime.strptime(txt_date1, "%d/%m/%Y")
    sale_week=dt2.isocalendar().week
    sale_year=dt2.year

  if st.button("Predict Weekly Sales", key="Weekly Sales"):
      # load the regression pickle model
      with open('/content/store_weekly_sale.pkl', 'rb') as f:
          model_store = pickle.load(f)

      user_data = np.array([[int(txt_store),int(sale_week),int(sale_year),int(week_holiday),np.log(float(txt_temp)),
                          float(txt_fuel_price),float(txt_CPI),np.log(float(txt_unemployment)),np.log(float(txt_Markdown1)),
                          np.log(float(txt_Markdown2)),np.log(float(txt_Markdown3)),
                          np.log(float(txt_Markdown4)),np.log(float(txt_Markdown5)),
                          ]])

      # model predict Sales based on user input
      y_pred = model_store.predict(user_data)

      Week_Sale = np.exp(y_pred[0])
      markdown=float(txt_Markdown1)+float(txt_Markdown2)+float(txt_Markdown3)+ float(txt_Markdown4)+float(txt_Markdown5)
      st.write("Store's Weekly Sale :", Week_Sale, "With Markdown",markdown)
      st.write("Expected sale price might be", Week_Sale+markdown)
with tab3:
    st.caption(":blue[Overview:]")
    st.caption(":blue[Store/Department weekly sales prediction]")
    st.caption(":blue[Data Cleaning has done for the all the null values ]")
    st.caption(":blue[Model built for Department sales prediction with 4 input features]")
    st.caption(":blue[Model built for general weekly sales prediction with markdown and holiday as input]")

In [ ]:
store_sales_df1

In [ ]:
!npm install localtunnel

In [ ]:
pip install streamlit

In [250]:
!streamlit run /content/app.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com